# V. Time series: pre-analysis

---
**Author(s):** Quentin Yeche, Kenji Ose, Dino Ienco - [UMR TETIS](https://umr-tetis.fr) / [INRAE](https://www.inrae.fr/)

---

## 1. Introduction

In this notebook we will introduce the time dimension into our exploration of data cubes. We will also cover handling masks, and specifically cloud cover masks.

## 2. Import libraries

As usual, we import all the required Python libraries.

In [ ]:
# STAC access
import pystac_client
import planetary_computer

# dataframes
import pandas as pd

# xarrays
import xarray as xr

# library for turning STAC objects into xarrays
import stackstac

# visualization
from matplotlib import pyplot as plt

# miscellanous
import numpy as np
from IPython.display import display
from datetime import date

## 3. Getting a Sentinel-2 images time series

### 3.1. Request on STAC catalog

As a practical use case let's consider that we have identified the STAC Collection we're interested in (see [this notebook](Joensuu_01-STAC.ipynb) for a refresher), and we also have an area of interest defined as a bounding box.

In [ ]:
aoi_bounds = (3.875107329166124, 43.48641456618909, 4.118824575734205, 43.71739887308995)

# retrieving the relevant STAC Item
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
    )

today = date.today()
last_month = today.replace(month=today.month-1).strftime('%Y-%m')
time_range = f"2020-01-01/{last_month}"
search = catalog.search(
    collections=['sentinel-2-l2a'],
    datetime=time_range,
    bbox=aoi_bounds
)
items = search.item_collection()
print(f"{len(items)} items found")

As a reminder, we display the assets description of the first item (or image):

In [ ]:
item = items[0]
assets = [asset.title for asset in item.assets.values()]
descriptions = pd.DataFrame(assets, 
                            columns=['Description'], 
                            index=pd.Series(item.assets.keys(), 
                                            name='asset_key'))
descriptions

<span style='color:red'> **IMPORTANT:** The URLs that we obtain from the STAC Catalog are not valid indefinitely. They expire after about 30 minutes. If you see an error in the notebook, it is likely because the URL that you obtained by running the first few cells has now expired.</span> If that happens you should be able to just run the notebook again from the top to get a new URL. You can get longer-lasting URLs by signing up for Planetary Computer (which is free at the time of writing this notebook). More info [here](https://planetarycomputer.microsoft.com/docs/concepts/sas/). 

### 3.2. Creating a datacube from STAC search results

This process is very similar to what we did to create a DataArray in the beginning of the [previous notebook](Joensuu_03-Xarray.ipynb).

In [ ]:
bands = ['B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B11', 'B12']
FILL_VALUE = 2**16-1
array = stackstac.stack(
                    items,
                    assets = bands,
                    resolution=10,
                    dtype="uint16",
                    fill_value=FILL_VALUE,
                    bounds_latlon=aoi_bounds,
                    )
array 

We can see that in a few lines of code we have created a DataArray which data has a size in the tens of gigabytes. The idea of lazy computations and not downloading data until necessary starts to make a lot more sense with these sizes. When we also consider the fact our area of interest is only about 5% of a full Sentinel-2 tile, the ability to download only part of a tile also becomes very important. If we had to download the entire tiles before creating this array the volume of data would probably be measured in terabytes!

### 3.3. Checking for invalid values

Before moving on, it is always a good idea to check the validity of the DataArray we just created. There are two main things to check for:
 - The presence of fill values. As described in the [previous notebook](Joensuu_03-Xarray.ipynb), the fill value is used by `stackstac.stack` in order to replace missing STAC Assets.
 - The presence of NODATA values. In our case (Sentinel-2) pixel values of 0 are used to flag that the pixel does not contain valid data.

#### 3.3.1. Checking the presence of fill values

Of course, checking for these values necessitate to download the data. Thus we can really only look at a sample.

In [ ]:
# we're only taking the first 4 dates
sample = array.isel(time=slice(0,4))
fill_values = xr.where(sample == FILL_VALUE, 1, 0).sum().values

print(f"Data contains {fill_values} fill value pixels")


As expected we see no fill values. Indeed we have no missing Asset in any of our Items. Our collection of STAC Items is homogenous, all Sentinel-2 Items contain the right Assets.

#### 3.3.2. Checking the presence of NoData

There are two situations in which NODATA pixels can appear. The first is extents which do not strictly overlap. Satellite images are almost always distributed as multi-dimensional array with a rectangular shape. If the bounds do not exactly match then the image is completed with NODATA values to make up a rectangular shape.


<figure align="center">
  <img src="resources/extent_nodata.png" width="30%" alt="STAC Item diagram">
  <figcaption>Sentinel-2 image with NODATA values
  </figcaption>
</figure>

The second situation where NODATA values can appear is when some pixels are flagged as invalid despite being inside the boundaries of the area being captured by the captor. Typically the amount of such invalid pixels is very low.

In [ ]:
no_data = xr.where(sample == 0, 1, 0).sum().values
print(f"{no_data/sample.size:.1%} of values are NODATA")

We can see that almost half of the pixels in the sample are invalid. So it appears that at least some our images do not fully overlap with our area of interest.

#### 3.3.3. Checking through `xarray` coordinates

The first thing we can look at is the `s2:nodata_pixel_percentage`. It gives us the percentage of pixels with NODATA in the tile for each date.

<span style='color:red'> **IMPORTANT:**</span> Planetary Computer provides a few metadata properties along with the data.
 For Sentinel-2 the most useful include:
 - `s2:nodata_pixel_percentage`
 - `s2:water_percentage`
 - `s2:vegetation_percentage`
 - `eo:cloud_cover`
 - `s2:high_proba_clouds_percentage`
 - `s2:cloud_shadow_percentage`
 
However <span style='color:red'> **these values were computed from the whole tile (an area of up to 110 x 110 km²), and they are NOT updated when only a sub-section of a tile is considered or downloaded**</span>. The values can still be used as a proxy or rough estimate without having to download the data. It is particularly useful when gathering data in order to decide what to keep or throw out. But be careful not to confuse them with the actual value which can be computed from a sub-section of a tile. For instance a Sentinel-2 tile could have a low value for `eo:cloud_cover`, indicating that there are relatively few clouds in the whole tile. But that doesn't mean that our area of interest could not contain some of these few clouds, leading to unusable data.


In [ ]:
[f'{pct:.2f} %' for pct in sample['s2:nodata_pixel_percentage']]

Half the dates have more than 85% of their pixels labelled as NODATA, while the others have few to none. Let's see if this behavior applies to the whole data cube:

In [ ]:
array['s2:nodata_pixel_percentage'].plot.hist()
plt.title('nodata pixel percentage')
plt.show()

It appears that we are dealing with two different types Sentinel-2 images, one of which only has a small amount of valid values. We can know more by looking at the `sat:relative_orbit` coordinate of our array. It gives us the number of the orbit during which the image was captured:

In [ ]:
array['sat:relative_orbit']

The images were captured with two different orbits: 8 and 108. Let's plot the same histogram as before, but discriminating between the two orbits:

In [ ]:
array.sel(time=array.time['sat:relative_orbit']==8)['s2:nodata_pixel_percentage'].plot.hist()
array.sel(time=array.time['sat:relative_orbit']==108)['s2:nodata_pixel_percentage'].plot.hist()

plt.legend(['Orbit 8', 'Orbit 108'])
plt.title('NODATA values percentage per orbit')
plt.show()

As suspected, one of the two orbits is mainly at fault: orbit 108. However, let's keep in mind that these values we're looking at from `s2:nodata_pixel_percentage` are for the whole image which corresponds to a much bigger area than ours. But at this point there is little chance that our area happens to overlap with this small portion of valid pixels.

We can check it with a similar test to what we did for the fill values:

In [ ]:
# selecting from our sample the dates with orbit 108
sample_108 = sample.sel(time=sample.time['sat:relative_orbit']==108)

# counting the number of NODATA pixels as a percentage
# of overall pixel count
nodata_108 = xr.where(sample_108==0,1,0).sum().values
print(f"{nodata_108/sample_108.size:.1%} of pixels in the array are NODATA")

Our area of interest falls almost entirely outside of the area of validity for the images with orbit 108. Thus we need to simply restrict our study to orbit 8.

In [ ]:
array = array.sel(time=array.time['sat:relative_orbit']==8)
array

In the previous histograms we could see that even with orbit 8 there were a few values with higher than average amounts of invalid pixels:

In [ ]:
array.sel(time=array.time['s2:nodata_pixel_percentage']>5)

Feel free to take a look at these dates and their data to decide whether they should be kept. Let's just decide to remove them, as this is only 3 dates out.

In [ ]:
array = array.sel(time=array.time['s2:nodata_pixel_percentage'] <5)
array

#### 3.3.4. Extra: The relationship between Sentinel-2 tiles and orbits

Sentinel-2 uses the concept of tiles. A Sentinel-2 tile delimits an area of 100x100 km². Each tile has a unique code. The tile that our area of interest falls in is the tile 31TEJ (often times the T is added in front: T31TEJ). 

**Note:** As an aside this code can be found in `s2:mgrs_tile` and as part of the `s2:granule_id` coordinate of our array. More information on the naming conventions used by Sentinel-2 can be found [here](https://sentinels.copernicus.eu/web/sentinel/user-guides/sentinel-2-msi/naming-convention#:~:text=The%20top%2Dlevel%20SENTINEL%2D2,separated%20by%20an%20underscore%20(_).&text=The%20Mission%20Identifier%20(MMM)%20denotes,for%20the%20SENTINEL%2D2B%20instrument.).

In order to capture an image in the area delimited by the tile 31TEJ the satellite has to fly over the area along an orbit. For tile 31TEJ this orbit is orbit 8. But in order to ensure that there exists overlap between the tiles, data is actually captured with some overlap in a 110x110 km² area. 

Here is now the explanation for the two orbits. The images we see with orbit 108 actually correspond to the acquisition for a different, neighboring tile 31TJG. But since there is overlap between the two images, a small area near the edge of tile 31TEJ is covered by the acquisitions made with orbit 108.

**Note:** A useful interactive map of the Sentinel-2 tiling grid is available [here](https://eatlas.org.au/data/uuid/f7468d15-12be-4e3f-a246-b2882a324f59) (it can take a few seconds for the map to load in completely). It clearly shows the different tiles, their codes and the overlap between them.

## 4. Operations on the time dimension

In the [previous notebook](Joensuu_03-Xarray.ipynb) we covered how to manipulate Xarrays. Now that we have introduced multiple dates in our time dimension, there are a few more tools which are useful when dealing with temporal data.

### 4.1. Pandas dates and inexact matching

There are situations in which having to match the exact values is not desirable. Date objects are usually given with a high degree of precision, and it is not practical having to always specify hours minutes and seconds in situations where the date would suffice. There are a few tools which help alleviate this annoyance.

The first is that Xarray uses the same logic as pandas for its date indexes. Even though the values are datetime64 objects, they can be selected with regular strings with a lot of convenience. For instance "`2023`" will correctly match all datetime64 objects between `2023-01-01T00:00:00` and `2023-12-31T23:59:59`, and `2023-01` will correctly match all datetime64 objects of the January 2023, etc.

The other tool is the ability of the `sel` method to select inexact matches. This behavior is not limited to time dimensions, but it is particularly applicable for dates. By default, `sel` only matches exact values. However the `method` parameter can enable inexact matches:
 - `pad` or `ffill` will match the nearest value below
 - `backfill` or `bfill` will match the nearest value above
 - `nearest` will match the nearest value

 Another parameter of `sel` is `tolerance`. It allows to set a limit on the distance between the specified value and a possible match. For dates it can used with a `timedelta64` objet or a string like `'5D'` (5 days), `'5M3W'` (5 months 3 weeks), `'2Y15h34s'`(2 years 15 hours and 34 seconds), etc.


In [ ]:
few_dates = array.isel(time=slice(3,7)).time
print(f"Dates:\n {few_dates.values}")

# does not match the last date as it is in February
print(f"\n2020-01: \n {few_dates.sel(time='2020-01').values}")

# matches 2020-01-18 as it is the nearest
print(f"\nnearest 2020-01-20 \n",
      f"{few_dates.sel(time='2020-01-20', method='nearest').values}")

# matches 2020-01-23 as it is the nearest date after 2020-01-20
print(f"\nbackfill 2020-01-20\n",
      f"{few_dates.sel(time='2020-01-20', method='backfill').values}")

# matches 2020-01-18 as it is the nearest date before 2020-01-22
print(f"\npad 2020-01-22 \n",
      f"{few_dates.sel(time='2020-01-22', method='pad').values}")

# 2020-01-23 is treated as 2020-01-23T00:00:00
# which comes before 2020-01-23T10:42:29.0240
# thus 2020-01-18 is the correct match
print(f"\npad 2023-01-23\n",
      f"{few_dates.sel(time='2020-01-23', method='pad').values}")

print(f"\npad 2023-01-23 with tolerance 4 days 14 hours\n",
      f"{few_dates.sel(time='2020-01-23', method='pad', tolerance='4D14h').values}")

# this one would fail to get a match and give an error
#print(f"\npad 2023-01-22 with tolerance 4 days 13 hours and 5 seconds\n",
#      f"{few_dates.sel(time='2020-01-23', method='pad', tolerance='4D13h5s').values}")

Another thing inherited from pandas is the `dt` accessor. It allows accessing specific components of a datetime64 object. The full list of components can be found [here](https://pandas.pydata.org/docs/user_guide/timeseries.html#time-date-components).

In [ ]:
# all the data from February for all the years
print(array.sel(time=(array.time.dt.month == 2)).time)
print('\n===\n')
# all the data captured on the 4th day of a month
print(array.sel(time=(array.time.dt.day == 4)).time)

### 4.2. Resampling and other operations
#### 4.2.1. Analysing acquisition frequency

What follows are a few examples of operations on dates in order to extract useful information. They can be instrumental in refining a time series. It is technically possible to do more complex operations like upsampling and interpolation, but they will not be covered here.

First, we create a funtion to to format axes for dates as a fairly readable year/month standard:

In [ ]:

def date_format(ax, month='%B'):
    import matplotlib.dates as mdates
    # adding locations and formatting for months
    # on the x axis
    loc = mdates.MonthLocator(bymonth=(1,4,7,10))
    major_fmt = mdates.ConciseDateFormatter(loc,
                formats=['%Y', month, '%d', '%H:%M', '%H:%M', '%S.%f']
                                            )
    ax.xaxis.set_major_formatter(major_fmt)
    ax.xaxis.set_major_locator(loc)

    # adding minor ticks for months without a label
    ax.xaxis.set_minor_locator(mdates.MonthLocator())

Then we calculate and plot the number of days between each acquisition to see if there any gaps:

In [ ]:
fig, ax = plt.subplots(figsize=(12,1))
ax.eventplot(array.time)
ax.set_title("Chronogram of acquisitions")

date_format(ax, month='%b')

# removing y axis ticks as they are meaningless
ax.set_yticks([])
fig.show()

# plotting as a histogram
fig, (ax1, ax2) = plt.subplots(2,1, sharex=True)
ax1.hist(array.time.diff(dim='time').dt.round('1D').dt.days)
ax2.hist(array.time.diff(dim='time').dt.round('1D').dt.days)
fig.subplots_adjust(hspace=0.05)
ax1.set_ylim(150,250)
ax2.set_ylim(0,16)

ax1.spines.bottom.set_visible(False)
ax2.spines.top.set_visible(False)
ax1.xaxis.tick_top()
ax2.xaxis.tick_bottom()

# adding cut-out slanted lines to signify a cut in the y axis
d = .5
kwargs = dict(marker=[(-1, -d), (1, d)], markersize=12,
              linestyle="none", color='k', mec='k', mew=1, clip_on=False)
ax1.plot([0, 1], [0, 0], transform=ax1.transAxes, **kwargs)
ax2.plot([0, 1], [1, 1], transform=ax2.transAxes, **kwargs)

ax1.set_title('Number of days between two acquisitions')
ax2.set_xlabel('Number of days')
# supylabel allows us to name both y axes together
fig.supylabel("Number of occurences")
plt.show()

fig.show()


We also can count the number of acquisitions per month:

In [ ]:
# resampling and counting for each month
# using MS (Month Start) as we prefer using the first day
# of the month as a reference point instead of the last
# acquisition date of the month
ar = array.time.resample(time='MS').count()

import matplotlib.dates as mdates
fig, ax = plt.subplots(figsize=(13,4))
ar.plot.scatter(ax=ax, x='time')
ax.set_ylabel('Acquisition count')
ax.set_title("Number of acquisitions per month")

# setting y ticks manually to avoid non-integer values being used
ax.set_yticks(range(ar.min().values, ar.max().values+1))

date_format(ax)
fig.show()

#### 4.2.2. Analysing variations of cloud cover

In [ ]:

# plotting the average cloud cover and vegetation for the acquistion for each month
# they are very negatively correlated since 'cloudy' and 'vegetation' 
# are two exclusive classes for a pixel
fig, ax = plt.subplots(figsize=(10,5))
array['eo:cloud_cover'].resample(time='M').mean().plot(ax=ax, color='slategrey')
#array['s2:vegetation_percentage'].resample(time='M').mean().plot(ax=ax, color='forestgreen')
ax.legend(['cloud cover'])#, 'vegetation'])
ax.set_ylabel('percentage')
ax.set_title('Average cloud cover percentage in the tile per month')

date_format(ax)

plt.show()